## Mathematics for AI - Coursework Task 1

The first task tests your Python skills. You need to develop a simple game consisting of a rectangular grid (of size height x width ) where each cell has a random integer value between 0  and 9. An agent starts at the upper-left corner of the grid and must reach the lower-right corner of the grid as fast as possible. You can implement one of the two (or both, for no extra point) game modes:

• The time spent on a cell is the number on this cell

• The time spent on a cell is the absolute of the difference between the previous cell the agent was on and the current cell it is on.

In order to solve this problem, you will provide 3 algorithms:

• A first baseline of your choosing. E.g. it can be any search algorithm, or an algorithm using heuristics. It doesn’t have to perform fast or well, but should be better than random movements.

• Dijkstra'salgorithm

• Ant colony optimization algorithm. You should describe the algorithms and compare them.

Are they always solving the problem? How long do they take depending on the size of the maze?

## Make Grid

In [6]:
# Import (limited) libraries
from random import seed
from random import randint
import numpy as np
import sys 

In [7]:
g = []
length = 10
seed(9)

for row in range(length):
    g.append([])
    for column in range(length):
        g[row].append(randint(0,9))

In [8]:
# Convert to numpy.array
grid = np.array(g)
print(grid)

[[7 9 5 4 2 2 0 5 8 7]
 [9 1 5 8 9 0 6 2 7 6]
 [2 2 3 0 1 2 8 9 1 6]
 [1 4 3 3 6 1 4 3 6 4]
 [5 0 3 0 6 0 6 7 2 0]
 [3 6 1 9 0 1 9 3 3 5]
 [0 1 2 8 0 8 1 9 7 8]
 [3 6 1 6 3 1 9 2 2 9]
 [0 0 4 8 9 2 4 9 0 1]
 [6 3 2 9 8 0 5 8 9 9]]


In [51]:
src = grid[0][0]
location = [(0,0)]
print(grid[])


TypeError: can only concatenate tuple (not "int") to tuple

## Baseline Algorithm

In [53]:
class Baseline():
    
    def __init__(self, graph):
        self.graph = graph
        
    def algorithm(self):
        
        visited = []
        moveTo
        src = [0,0]
        finish = [length-1,length-1]
        current = src
        
        while current != finish:
            visited.append(current)
            X = current[1]
            y = current[0]
            locations = [(y, X + 1),(y, X - 1), (y + 1, X),(y - 1, X)]
            print(locations)
            
            
        

In [117]:
visited = []
src = [0,0]
finish = [length-1,length-1]
current = src
timeSpent = 0
        
while current != finish:
    visited.append(current)
    print(visited)
    values = []
    X = current[1]
    y = current[0]
    timeSpent += grid[X,y]
    locations = [[y, X + 1],[y, X - 1], [y + 1, X],[y - 1, X]]
    for i in locations:
        if(i[0] < 0 or i[1] < 0 or i[0] > length - 1 or i[1] > length - 1):
            locations.remove(i)
    
    for j in locations:
        if(j in visited):
            locations.remove(j)
            
    for k in locations:
        value = grid[k[0], k[1]]
        values.append(value)
      
    final = dict(zip(values, locations))
    moveTo = 0
    for l in sorted (final.keys()):  
        print(l)
        moveTo = l
            
    current = final[moveTo]

[[0, 0]]
9
[[0, 0], [1, 0]]
1
2
[[0, 0], [1, 0], [2, 0]]
1
2
[[0, 0], [1, 0], [2, 0], [2, 1]]
1
3
4
[[0, 0], [1, 0], [2, 0], [2, 1], [3, 1]]
0
1
3
[[0, 0], [1, 0], [2, 0], [2, 1], [3, 1], [3, 2]]
3
[[0, 0], [1, 0], [2, 0], [2, 1], [3, 1], [3, 2], [2, 2]]
0
3
5
[[0, 0], [1, 0], [2, 0], [2, 1], [3, 1], [3, 2], [2, 2], [1, 2]]
1
5
8
[[0, 0], [1, 0], [2, 0], [2, 1], [3, 1], [3, 2], [2, 2], [1, 2], [1, 3]]
0
4
9
[[0, 0], [1, 0], [2, 0], [2, 1], [3, 1], [3, 2], [2, 2], [1, 2], [1, 3], [1, 4]]
0
1
2
[[0, 0], [1, 0], [2, 0], [2, 1], [3, 1], [3, 2], [2, 2], [1, 2], [1, 3], [1, 4], [0, 4]]
2
4
[[0, 0], [1, 0], [2, 0], [2, 1], [3, 1], [3, 2], [2, 2], [1, 2], [1, 3], [1, 4], [0, 4], [0, 3]]
5
[[0, 0], [1, 0], [2, 0], [2, 1], [3, 1], [3, 2], [2, 2], [1, 2], [1, 3], [1, 4], [0, 4], [0, 3], [0, 2]]
9
[[0, 0], [1, 0], [2, 0], [2, 1], [3, 1], [3, 2], [2, 2], [1, 2], [1, 3], [1, 4], [0, 4], [0, 3], [0, 2], [0, 1]]
1
7
[[0, 0], [1, 0], [2, 0], [2, 1], [3, 1], [3, 2], [2, 2], [1, 2], [1, 3], [1, 4], [0, 4

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



KeyboardInterrupt: 

## Dijkstra's Algorithm

In [10]:
import sys 
   
class Graph(): 
   
    def __init__(self, vertices): 
        self.V = vertices 
        self.graph = [[0 for column in range(vertices)]  
                    for row in range(vertices)] 
   
    def printSolution(self, dist): 
        print ("Vertex tDistance from Source") 
        for node in range(self.V): 
            print (node, "t", dist[node]) 
   
    # A utility function to find the vertex with  
    # minimum distance value, from the set of vertices  
    # not yet included in shortest path tree 
    def minDistance(self, dist, sptSet): 
   
        # Initilaize minimum distance for next node 
        min = sys.maxsize 
   
        # Search not nearest vertex not in the  
        # shortest path tree 
        for v in range(self.V): 
            if dist[v] < min and sptSet[v] == False: 
                min = dist[v] 
                min_index = v 
   
        return min_index 
   
    # Funtion that implements Dijkstra's single source  
    # shortest path algorithm for a graph represented  
    # using adjacency matrix representation 
    def dijkstra(self, src): 
   
        dist = [sys.maxsize] * self.V 
        dist[src] = 0
        sptSet = [False] * self.V 
   
        for cout in range(self.V): 
   
            # Pick the minimum distance vertex from  
            # the set of vertices not yet processed.  
            # u is always equal to src in first iteration 
            u = self.minDistance(dist, sptSet) 
   
            # Put the minimum distance vertex in the  
            # shotest path tree 
            sptSet[u] = True
   
            # Update dist value of the adjacent vertices  
            # of the picked vertex only if the current  
            # distance is greater than new distance and 
            # the vertex in not in the shotest path tree 
            for v in range(self.V): 
                if self.graph[u][v] > 0 and sptSet[v] == False and dist[v] > dist[u] + self.graph[u][v]: 
                    dist[v] = dist[u] + self.graph[u][v] 
   
        self.printSolution(dist) 
    
g = Graph(length) 
g.graph = grid
   
g.dijkstra(0); 

Vertex tDistance from Source
0 t 0
1 t 5
2 t 3
3 t 4
4 t 2
5 t 2
6 t 8
7 t 5
8 t 4
9 t 5


## Ant Colony Algorithm